# Run framewise displasment

In [ ]:
import glob
import os
import sys
import subprocess
 
path = '/mnt/d/Data/Pigeon/analysis/ColorDisc/'
 
bold_files = glob.glob('%s/sub*/*/func.nii.gz'%(path))

# I'm using a big html file to put all QA info together.  If you have other suggestions, let me know!
outhtml = "/mnt/d/Data/Pigeon/analysis/ColorDisc/Scripts/bold_motion_QA.html"
out_bad_bold_list = "/mnt/d/Data/Pigeon/analysis/ColorDisc/Scripts/subs_lose_gt_45_vol_scrub.txt"
 
os.system("rm %s"%(out_bad_bold_list))
os.system("rm %s"%(outhtml))
 
for cur_bold in list(bold_files):
    #print(cur_bold)
     # Store directory name
    cur_dir = os.path.dirname(cur_bold)
    print(cur_dir)
    # strip off .nii.gz from file name (makes code below easier)
    #cur_bold_no_nii = cur_bold[:-7]
    #print(cur_bold_no_nii)
    #  You can also use fslreorient2std BUT
    #  BUT BUT BUT DO NOT RUN THIS UNLESS YOUR DATA ACTUALLY NEED IT!
    #os.system("fslswapdim %s z -x -y %s_swapped"%(cur_bold_no_nii, cur_bold_no_nii))
    # Once you're confident this works correctly, you can change the above to
    #  overwrite bold.nii.gz (saves disc space)
 
    # This is used to trim off unwanted volumes
    # DO NOT USE THIS UNLESS YOU'VE DOUBLE CHECKED HOW MANY
    # VOLUMES NEED TO BE TRIMMED (IF ANY)
    # This trims first 2 and I set the max to a number far beyond
    # the number of TRs
    # Correct filename here to use output of previous step (if used)
    #os.system("fslroi %s %s_trimmed 2 300"%(cur_bold_no_nii, cur_bold_no_nii))
    # Once you're confident this works correctly, you can change the above to
    #   overwrite bold.nii.gz
 
    # Assessing motion.  This is what takes the longest
    # Check current literature to see if this thresh (0.9) is
    #  acceptable
    # I got it from here: http://www.ncbi.nlm.nih.gov/pubmed/23861343
    # Also, consider using FSL's FIX to clean your data
    output_path = os.path.join(cur_dir,'motion_assess')
    if os.path.isdir("%s"%(output_path))==False:
        os.makedirs(output_path)

    os.system("fsl_motion_outliers -i %s -o %s/confound.txt --fd --thresh=2 -p %s/fd_plot -s %s/fd_par.txt -v > %s/outlier_output.txt"%(cur_bold, output_path, output_path, output_path, output_path))
    # Put confound info into html file for review later on
    os.system("cat %s/outlier_output.txt >> %s"%(output_path, outhtml))
    os.system("echo '<p>=============<p>FD plot %s <br><IMG BORDER=0 SRC=%s/fd_plot.png WIDTH=100%s></BODY></HTML>' >> %s"%(output_path, output_path,'%', outhtml))
 
    # Last, if we're planning on modeling out scrubbed volumes later
    #   it is helpful to create an empty file if confound.txt isn't
    #   generated (i.e. no scrubbing needed).  It is basically a
    #   place holder to make future scripting easier
    if os.path.isfile("%s/confound.txt"%(output_path))==False:
        os.system("touch %s/confound.txt"%(output_path))
 
    # Very last, create a list of subjects who exceed a threshold for
    #  number of scrubbed volumes.  This should be taken seriously.  If
    #  most of your scrubbed data are occurring during task, that's
    #  important to consider (e.g. subject with 20 volumes scrubbed
    #  during task is much worse off than subject with 20 volumes
    #  scrubbed during baseline.
    # These data have about 182 volumes and I'd hope to keep 140
    #  DO NOT USE 140 JUST BECAUSE I AM.  LOOK AT YOUR DATA AND
    #  COME TO AN AGREED VALUE WITH OTHER RESEARCHERS IN YOUR GROUP
    output = subprocess.check_output("grep -o 1 %s/confound.txt | wc -l"%(output_path), shell=True)
    num_scrub = [int(s) for s in output.split() if s.isdigit()]
    if num_scrub[0]>45:
        with open(out_bad_bold_list, "a") as myfile:
            myfile.write("%s\n"%(cur_bold))

/mnt/d/Data/Pigeon/analysis/ColorDisc/sub01/acquisition
/mnt/d/Data/Pigeon/analysis/ColorDisc/sub01/ext_day1
